In [195]:
# load the packages 
import pickle
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import scipy.sparse

# Load required pkl and csv

In [ ]:
group1 = pd.read_csv("FAERS_columnSubset_Group_1.csv")
group2 = pd.read_csv("FAERS_columnSubset_Group_2.csv")
group3 = pd.read_csv("FAERS_columnSubset_Group_3.csv")
group4 = pd.read_csv("FAERS_columnSubset_Group_4.csv")
group5 = pd.read_csv("FAERS_columnSubset_Group_5.csv")
group6 = pd.read_csv("FAERS_columnSubset_Group_6.csv")
group7 = pd.read_csv("FAERS_columnSubset_Group_7.csv")
group8 = pd.read_csv("FAERS_columnSubset_Group_8.csv")
group9 = pd.read_csv("FAERS_columnSubset_Group_9.csv")
group10 = pd.read_csv("FAERS_columnSubset_Group_10.csv")

In [197]:
# load the unique_dict_categorical file for unique values for the feature matrix 
file_name = open("unique_categorical_dict.pkl", "r")
unique_categorical_dict = pickle.load(file_name)
file_name.close()

In [198]:
# load the  file for unique values for the reaction, create both a dictionary and a list. 
top2000Reactions = pd.read_csv("Top2000Reactions.csv")
unique_react_list = top2000Reactions['Reaction'].unique()
reactions_set = set(top2000Reactions.Reaction)

# OneHotEncoding and split the feature matrix(X) and reactions(Y)

In [199]:
#Create function to convert to dummy variables
def convert_to_dummies(data, categorical_columns, unique_dictionary):
    #Assign the categories for each column
    for column in categorical_columns:
        data[column] = data[column].astype('category', categories=unique_dictionary[column])
    #Convert to categorical
    dummy = pd.get_dummies(data, columns=categorical_columns)
    return dummy

In [200]:
# Code to convert to dummy a specific group, only the feature matrix, the reaction will keep the same 
categorical_columns = ['RxCUI', 'indications', 'role_cod', 'sex', 'age_grp', 
                       'dechal', 'rechal']
group1_dummy = convert_to_dummies(group1, categorical_columns, unique_categorical_dict)
group2_dummy = convert_to_dummies(group2, categorical_columns, unique_categorical_dict)
group3_dummy = convert_to_dummies(group3, categorical_columns, unique_categorical_dict)
group4_dummy = convert_to_dummies(group4, categorical_columns, unique_categorical_dict)
group5_dummy = convert_to_dummies(group5, categorical_columns, unique_categorical_dict)
group6_dummy = convert_to_dummies(group6, categorical_columns, unique_categorical_dict)
group7_dummy = convert_to_dummies(group7, categorical_columns, unique_categorical_dict)
group8_dummy = convert_to_dummies(group8, categorical_columns, unique_categorical_dict)
group9_dummy = convert_to_dummies(group9, categorical_columns, unique_categorical_dict)
group10_dummy = convert_to_dummies(group10, categorical_columns, unique_categorical_dict)

In [201]:
# Separate select the reactions columns to create Y_value
def separate_x_y(data):
    data_x = data[data.columns.difference(['Unnamed: 0','Unnamed: 0.1','caseid','drug_seq','reactions'])]
    data_y =data.filter(regex='reactions')
    return data_x,data_y

In [202]:
# create the oneHotEncoding data
group1_x,group1_y = separate_x_y(group1_dummy)
group2_x,group2_y = separate_x_y(group2_dummy)
group3_x,group3_y = separate_x_y(group3_dummy)
group4_x,group4_y = separate_x_y(group4_dummy)
group5_x,group5_y = separate_x_y(group5_dummy)
group6_x,group6_y = separate_x_y(group6_dummy)
group7_x,group7_y = separate_x_y(group7_dummy)
group8_x,group8_y = separate_x_y(group8_dummy)
group9_x,group9_y = separate_x_y(group9_dummy)
group10_x,group10_y = separate_x_y(group10_dummy)

# Reshape both the X and Y, Save all as Pickled Files

In [203]:
# Reshape the Feature Matrix 
# the funcation to reshape the data format, size is the number of the group, group is the data
def splitX(size, group):
    caseid_data = []
    group_size = len(group)/size
    for case_order in range(group_size):
        row = group.loc[(case_order*size):(case_order*size+(size-1))].values
        newrow = scipy.sparse.csc_matrix(row)
        caseid_data.append(newrow)
    return caseid_data

def split_coma(s):
    return s.split('\\,')

#Remove the reactions that are not in the set of 2000
def reaction_intersection(reaction_list):
    return list(reactions_set.intersection(reaction_list))

def splitY(size,group):
    reactions_splited = pd.DataFrame()
    reactions_splited['reaction'] = map(split_coma, group10_y['reactions'])
    y_splited = pd.DataFrame()
    y_splited['reaction'] = map(reaction_intersection,reactions_splited.reaction)
    mlb= MultiLabelBinarizer(classes=unique_react_list)
    group_formated = mlb.fit_transform(y_splited['reaction'])
    caseid_data = []
    group_size = len(group_formated)/size
    for case_order in range(group_size):
        row = group_formated[(case_order*size):(case_order*size+size)]
        newrow = scipy.sparse.csc_matrix(row)
        caseid_data.append(newrow)
    return caseid_data

def savePickledFile(name,data):
    with open (name,"wb") as fp:
        pickle.dump(data,fp)

In [ ]:
# Save all the feature matrix for different groups into pickled File 
group1_x_sparse = splitX(1,group1_x)
# savePickledFile("group1_x_sparse.pkl",group1_x_sparse)
group1_y_sparse = splitY(1,group1_y)
savePickledFile("group1_y_sparse.pkl",group1_y_sparse)    

group2_x_sparse = splitX(2,group2_x)
savePickledFile("group2_x_sparse.pkl",group2_x_sparse)
group2_y_sparse = splitY(2,group2_y)
savePickledFile("group2_y_sparse.pkl",group2_y_sparse)  

group3_x_sparse = splitX(3,group3_x)
savePickledFile("group3_x_sparse.pkl",group3_x_sparse)
group3_y_sparse = splitY(3,group3_y)
savePickledFile("group3_x_sparse.pkl",group3_x_sparse)  

group4_x_sparse = splitX(4,group4_x)
savePickledFile("group4_x_sparse.pkl",group4_x_sparse)
group4_y_sparse = splitY(4,group4_y)
savePickledFile("group4_y_sparse.pkl",group4_y_sparse)  

group5_x_sparse = splitX(5,group5_x)
savePickledFile("group5_x_sparse.pkl",group5_x_sparse)
group5_y_sparse = splitY(5,group5_y)
savePickledFile("group5_y_sparse.pkl",group5_y_sparse)  

group6_x_sparse = splitX(6,group6_x)
savePickledFile("group6_x_sparse.pkl",group6_x_sparse)
group6_y_sparse = splitY(6,group6_y)
savePickledFile("group6_y_sparse.pkl",group6_y_sparse) 

group7_x_sparse = splitX(7,group7_x)
savePickledFile("group7_x_sparse.pkl",group7_x_sparse)
group7_y_sparse = splitY(7,group7_y)
savePickledFile("group7_y_sparse.pkl",group7_y_sparse) 

group8_x_sparse = splitX(8,group8_x)
savePickledFile("group8_x_sparse.pkl",group8_x_sparse)
group8_y_sparse = splitY(8,group8_y)
savePickledFile("group8_y_sparse.pkl",group8_y_sparse) 

group9_x_sparse = splitX(9,group9_x)
savePickledFile("group9_x_sparse.pkl",group9_x_sparse)
group9_y_sparse = splitY(9,group9_y)
savePickledFile("group9_y__sparse.pkl",group9_y_sparse)

group10_x_formated = splitX(10,group10_x)
savePickledFile("group10_x_formated.pkl",group10_x_formated)
group10_y_formated = splitY(10,group10_y)
savePickledFile("group10_y_formated.pkl",group10_y_formated)

In [192]:
print(group10_x_sparse[2].todense().shape)
print(group10_y_sparse[2].todense().shape)

(10L, 9321L)
(10L, 2000L)


In [193]:
print(group10_x_sparse[2].shape)
print(group10_y_sparse[2].shape)

(10, 9321)
(10, 2000)


In [116]:
#check header 
# print (list(mlb.classes_))
# print (mlb.inverse_transform(group_formated)[1])
# np.where(group_formated[1]==1)